## **PROJECT : Language_Translator**

In [1]:
#checking if GPU is running or not

!nvidia-smi

Fri Nov 14 05:36:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Install Required Libraries**

In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.3 MB/s eta 0:00:00


**Import Libraries**

In [3]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

**Model and Dataset Define**

In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

Helsinki-NLP/opus-mt-de-en Model

source:https://huggingface.co/Helsinki-NLP/opus-mt-de-en

The Dataset

source:https://huggingface.co/datasets/Aarif1430/english-to-hindi

In [5]:
 raw_datasets = load_dataset("Aarif1430/english-to-hindi")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-71c2cec7402cd4(…):   0%|          | 0.00/21.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127705 [00:00<?, ? examples/s]

**Dataset Output Check**

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['english_sentence', 'hindi_sentence'],
        num_rows: 127705
    })
})

In [7]:
raw_datasets['train'][1]

{'english_sentence': 'Whosoever desires the reward of the world, with Allah is the reward of the world and of the Everlasting Life. Allah is the Hearer, the Seer.',
 'hindi_sentence': 'और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है तो ख़ुदा के पास दुनिया व आख़िरत दोनों का अज्र मौजूद है और ख़ुदा तो हर शख्स की सुनता और सबको देखता है'}

**Data Preprocessing**

Tokenizer Load

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
tokenizer("Hello, this is a sentence!")

{'input_ids': [12110, 2, 90, 23, 19, 8800, 61, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

**Hindi Tokenization**

In [10]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(["और जो शख्स (अपने आमाल का) बदला दुनिया ही में चाहता है"]))

{'input_ids': [[9, 41, 1255, 17, 1788, 4783, 24, 13, 961, 463, 58, 11, 486, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Preprocessing Parameters

In [11]:
max_input_length = 128
max_target_length = 128

source_lang = "english_sentence" # Changed from "en" to match dataset key
target_lang = "hindi_sentence"   # Changed from "hi" to match dataset key

def preprocess_function(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    #setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"] # Fixed typo: inputs_ids -> input_ids
    return model_inputs

**Preprocessing Function**

In [12]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[1008, 2, 10919, 1411, 2, 66, 80, 6762, 137, 4650, 70, 16, 387, 30939, 14295, 2, 308, 253, 411, 79, 1257, 79, 4, 16930, 24351, 16, 573, 48, 1692, 7, 1102, 12370, 10535, 624, 10, 6467, 9275, 0], [9008, 2260, 4, 1120, 8, 4, 319, 2, 50, 107, 23, 4, 1120, 8, 4, 319, 10, 8, 4, 8370, 2932, 3, 107, 23, 4, 11510, 2, 4, 21492, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[8020, 6, 13400, 707, 314, 4188, 6, 116, 12222, 1044, 531, 8770, 680, 24998, 6538, 11, 42954, 428, 14300, 4553, 800, 169, 58, 6007, 1528, 248, 89, 5855, 11, 159, 8492, 25039, 9, 11656, 17514, 37, 44275, 82, 40, 0], [9, 41, 1255, 17, 1788, 4783, 24, 13, 961, 463, 58, 11, 486, 5, 35, 152, 6, 173, 463, 361, 44, 1627, 425, 24, 7023, 1033, 5, 9, 152, 35, 266, 1255, 15, 2937, 9, 2134, 2592, 5, 0]]}

Processing Apply

In [13]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/127705 [00:00<?, ? examples/s]

Model load

In [14]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Trainning Parameters

In [15]:
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

Data collator

In [16]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [17]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

Dataset prepare for training

In [18]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

optimizer setup

In [19]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

Madel training

GPU check with pytorch

In [ ]:
# TensorFlow models automatically utilize available GPUs if detected, so no explicit .to("cuda") call is needed.

In [ ]:
#model.fit(train_dataset, epochs=1)

KeyboardInterrupt: 

In [20]:
import torch
print(torch.cuda.is_available())


True


Small Dataset Training

In [21]:
small_train_dataset = raw_datasets["train"].select(range(500))  # sirf 500 samples


In [22]:
small_train_raw_dataset = raw_datasets["train"].select(range(500))
small_train_tokenized_dataset = small_train_raw_dataset.map(preprocess_function, batched=True)
small_train_tf_dataset = model.prepare_tf_dataset(
    small_train_tokenized_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)
model.fit(small_train_tf_dataset, epochs=1)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

31/31 [==============================] - 43s 303ms/step - loss: 4.4544


Model Save

In [23]:
model.save_pretrained("tf_model/")

/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:461: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


Model Testing


In [24]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [25]:
input_text = "where are you going?"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[61949    73  1746   209   160    51    22     0 61949]], shape=(1, 9), dtype=int32)


In [26]:
with tokenizer.as_target_tokenizer():
  print(tokenizer.decode(out[0], skip_special_tokens=True))

तुम कहाँ जा रहे हो?
